In [5]:
import pandas as pd
import re


In [6]:
def import_csv_file(file_path):
    try:
        df = pd.read_csv(file_path, on_bad_lines='skip')
        # delete unnecessary columns
        df = df.drop_duplicates().reset_index(drop=True)
        print(f"✅ データファイル '{file_path}' を正常に読み込みました。")
        return df
    except FileNotFoundError:
        print(f"❌ エラー: ファイルが見つかりません。パスを確認してください: '{file_path}'")
        return None

In [7]:
def create_accuracy_matrix(df: pd.DataFrame) -> list:
    accuracy_matrix = []
    current_list = []
    previous_seed = -1  # 初期値を -1 にしておく

    for _, row in df.iterrows():
        seed = int(row['dataset_seed'])  
        acc = row['テスト精度 (Test Acc)']

        if previous_seed != -1 and seed == 0:
            accuracy_matrix.append(current_list)
            current_list = [acc]
        else:
            current_list.append(acc)

        previous_seed = seed

    if current_list:
        accuracy_matrix.append(current_list)

    return accuracy_matrix


In [8]:
def compute_aa_and_aia(acc_matrix: list[list[float]]) -> tuple[list[float], list[float]]:
    """
    acc_matrix[k][j] = 第kタスク学習後の、第jタスクに対する精度 a_{k,j}
    を前提として、AA_kとAIA_kをそれぞれ計算する。
    
    Returns:
        AA_list: 各ステップにおけるAA_k（平均精度）
        AIA_list: 各ステップにおけるAIA_k（累積平均精度）
    """
    AA_list = []
    AIA_list = []

    for k in range(len(acc_matrix)):
        # k番目のタスクまで学習後の精度リスト: acc_matrix[k][0〜k]
        current_accs = acc_matrix[k][:k+1]  # j <= k
        AA_k = sum(current_accs) / len(current_accs)
        AA_list.append(AA_k)

        # AIA_k = 平均(AA_0 〜 AA_k)
        AIA_k = sum(AA_list) / len(AA_list)
        AIA_list.append(AIA_k)

    return AA_list, AIA_list

In [9]:
from typing import Tuple
def compute_latency_and_memory_stats(df: pd.DataFrame) -> Tuple[float, float, float]:
    """
    CSVに記録されたログから、以下の平均を計算する関数:
      - 推論時 平均レイテンシ
      - 訓練時 Memory Allocated [MB]
      - 推論時 Memory Allocated [MB]
    """

    latency_col = "平均レイテンシ (ms/バッチ)"
    train_mem_col = "訓練時 Memoey Allocated [MB]"
    #train_mem_res = "訓練時 Memory Reserved [MB]"
    infer_mem_col = "推論時 Memoey Allocated [MB]"
    #infer_mem_res = "推論時 Memory Reserved [MB]"

    # 数値のみを抽出し、NaNや無効値を除外して平均を計算
    latency_avg = df[latency_col].dropna().astype(float).mean()
    train_mem_avg = df[train_mem_col].dropna().astype(float).mean()
    #train_mem_res_avg = df[train_mem_res].dropna().astype(float).mean()
    infer_mem_avg = df[infer_mem_col].dropna().astype(float).mean()
    #infer_mem_res_avg = df[infer_mem_res].dropna().astype(float).mean()

    return latency_avg, train_mem_avg, infer_mem_avg

In [10]:
df= import_csv_file("/work/csv/DIL/random_baseline.csv")
acc_matrix= create_accuracy_matrix(df)
print(df)
print(compute_aa_and_aia(acc_matrix))
print(compute_latency_and_memory_stats(df))

✅ データファイル '/work/csv/DIL/random_baseline.csv' を正常に読み込みました。
   ステータス       モデル データセット  dataset_seed  LSTMCell optimizer      scheduler  \
0     完了  sequence  mnist             0      True     adamw  cosine_warmup   
1     完了  sequence  mnist             0      True     adamw  cosine_warmup   
2     完了  sequence  mnist             1      True     adamw  cosine_warmup   
3     完了  sequence  mnist             0      True     adamw  cosine_warmup   
4     完了  sequence  mnist             1      True     adamw  cosine_warmup   
5     完了  sequence  mnist             2      True     adamw  cosine_warmup   
6     完了  sequence  mnist             0      True     adamw  cosine_warmup   
7     完了  sequence  mnist             1      True     adamw  cosine_warmup   
8     完了  sequence  mnist             2      True     adamw  cosine_warmup   
9     完了  sequence  mnist             3      True     adamw  cosine_warmup   
10    完了  sequence  mnist             0      True     adamw  cosine_warmup   
11   

In [16]:
# --- リプレイベース実験のファイルリスト ---
replay_files_info = [
    {'label': 'ER mem=500, bs=8', 'mem': 500, 'bs': 8, 'path': '/work/csv/DIL/er_random/er_sweep_mem500_bs8.csv'},
    {'label': 'ER mem=500, bs=32', 'mem': 500, 'bs': 32, 'path': '/work/csv/DIL/er_random/er_sweep_mem500_bs32.csv'},
    {'label': 'ER mem=500, bs=128', 'mem': 500, 'bs': 128, 'path': '/work/csv/DIL/er_random/er_sweep_mem500_bs128.csv'},
    {'label': 'ER mem=3k, bs=8', 'mem': 3000, 'bs': 8, 'path': '/work/csv/DIL/er_random/er_sweep_mem3000_bs8.csv'},
    {'label': 'ER mem=3k, bs=32', 'mem': 3000, 'bs': 32, 'path': '/work/csv/DIL/er_random/er_sweep_mem3000_bs32.csv'},
    {'label': 'ER mem=3k, bs=128', 'mem': 3000, 'bs': 128, 'path': '/work/csv/DIL/er_random/er_sweep_mem3000_bs128.csv'},
    {'label': 'ER mem=5k, bs=8', 'mem': 5000, 'bs': 8, 'path': '/work/csv/DIL/er_random/er_sweep_mem5000_bs8.csv'},
    {'label': 'ER mem=5k, bs=32', 'mem': 5000, 'bs': 32, 'path': '/work/csv/DIL/er_random/er_sweep_mem5000_bs32.csv'},
    {'label': 'ER mem=5k, bs=128', 'mem': 5000, 'bs': 128, 'path': '/work/csv/DIL/er_random/er_sweep_mem5000_bs128.csv'},
    # ... 他のリプレイベースの結果もここに追加 ...
]

# 結果を格納するためのリスト
performance_summary = []

print("各CSVファイルを読み込み、パフォーマンス指標の平均値を計算します...")

for info in replay_files_info:
    file_path = info['path']
    
    try:
        df = pd.read_csv(file_path, on_bad_lines='skip')
    except FileNotFoundError:
        print(f"⚠️ ファイルが見つかりません: {file_path}")
        continue

    # 訓練時のログ（'モデル'列が空でない）のみを抽出
    train_logs = df[df['モデル'].notna()].drop_duplicates()
    # テスト時のログ（'テスト精度'列が空でない）を抽出
    test_logs = df[df['テスト精度 (Test Acc)'].notna()].drop_duplicates()

    # 各指標の平均値を計算 (値が存在しない場合は0とする)
    avg_train_mem = train_logs['訓練時 Memoey Allocated [MB]'].mean() if not train_logs.empty else 0
    avg_inference_mem = test_logs['推論時 Memoey Allocated [MB]'].mean() if not test_logs.empty else 0
    avg_latency = test_logs['平均レイテンシ (ms/バッチ)'].mean() if not test_logs.empty else 0
    
    performance_summary.append({
        'Experiment': info['label'],
        'Avg Train Memory (MB)': avg_train_mem,
        'Avg Inference Memory (MB)': avg_inference_mem,
        'Avg Latency (ms/batch)': avg_latency
    })

# 結果をDataFrameに変換して表示
summary_df = pd.DataFrame(performance_summary)

print("✅ 計算が完了しました。")
display(summary_df.round(2)) # 小数点第2位まで表示

各CSVファイルを読み込み、パフォーマンス指標の平均値を計算します...
✅ 計算が完了しました。


,Experiment,Avg Train Memory (MB),Avg Inference Memory (MB),Avg Latency (ms/batch)
0,"ER mem=500, bs=8",NaN,21.9,38.44
1,"ER mem=500, bs=32",NaN,21.9,38.38
2,"ER mem=500, bs=128",NaN,21.9,38.17
3,"ER mem=3k, bs=8",NaN,21.9,38.26
4,"ER mem=3k, bs=32",NaN,21.9,37.95
5,"ER mem=3k, bs=128",NaN,21.9,38.25
6,"ER mem=5k, bs=8",NaN,21.9,37.82
7,"ER mem=5k, bs=32",NaN,21.9,38.29
8,"ER mem=5k, bs=128",NaN,21.9,37.97
